In [1]:
!which python

/Users/guidosidoni/opt/anaconda3/envs/develop/bin/python


In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

In [177]:

df_c = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_d = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_r = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
def make_data(df):
    df.set_index(['Province/State', 'Country/Region', 'Lat', 'Long'], inplace=True)
    df = (df.
         stack().
        reset_index().
        rename(columns={'level_4':'issue_date', 0:'cant', 'Country/Region':'Country'})
    )
    df = df.groupby(['Country','issue_date']).sum().drop(['Lat','Long'], axis=1).reset_index()
    df.issue_date = pd.to_datetime(df.issue_date)
    df = df.set_index(['issue_date','Country']).unstack()
    df = df.loc[:, 'cant']
    return df

In [178]:
df_c = make_data(df_c)
df_d = make_data(df_d)
df_r = make_data(df_r)
df_rate = (df_d / df_c).fillna(0).replace(np.inf, 0)
df_act = df_c - df_d - df_r

In [179]:
import statsmodels.api as sm
def OLS_summary(x_data):
    x_data = temp.days.values
    X = sm.add_constant(x_data)
    mod = sm.OLS(np.log(y_data), X)
    res = mod.fit()
    print(res.summary())

In [180]:
import json
import requests
import numpy as np
import pandas as pd 

def daterange(start_date, N=5):
    #start_date = datetime.fromisoformat(start_date)
    start_date = date(int(start_date[:4]), int(start_date[6:7]), int(start_date[8:10]))
    for n in range(0, N):
        start_part = start_date + timedelta(days=n)
        yield start_part.strftime("%Y-%m-%d")



def get_covid19_data():
    alldata = []
    
    prefix_url = 'https://covidapi.hackfunrosario.com/api/getStatByFecha/'
    allresponse = []
    for date_report in daterange('2020-03-01', N=60):
        url = prefix_url + date_report
        response = requests.get(url)
        if len(allresponse) == 0:
            allresponse = pd.DataFrame(json.loads(response.text).get('data'))
        else:
            allresponse = pd.concat([allresponse, pd.DataFrame(json.loads(response.text).get('data'))])
    return allresponse

In [167]:
response = get_covid19_data()

In [175]:
response.provincia.unique()

array([None, 'CABA', 'Indeterminado', 'Buenos Aires', 'xx', 'Río Negro',
       'Chaco', 'San Luis', 'Tierra Del Fuego', 'Santa Cruz',
       'Entre Ríos', 'Salta', 'Jujuy', 'Córdoba', 'Santa Fe', 'Tucumán',
       'Santiago Del Estero', 'Neuquén', 'Corrientes', 'Mendoza',
       'Misiones', 'La Pampa', 'Tierra del Fuego', 'Santiago del Estero',
       'La Rioja', 'San Juan'], dtype=object)